In [56]:
# manual inputs
testing_status = True
# set start and end date
startdate_string = '2022-08-22'
entryduration = 7*4
# create new or not
update_Todoist = False

In [2]:
# import libaries
import os
import numpy as np
import pandas as pd
from todoist_api_python.api import TodoistAPI
import datetime

In [19]:
# api token file name
if testing_status:
    todoist_token_fname = 'todoist_api_token_test.txt'
    fname_inputcsv = 'todoist-python - Routine-test.csv'
    fpath_inputcsv = os.path.join(os.getcwd(), fname_inputcsv)
else:
    todoist_token_fname = 'todoist_api_token.txt'
    fname_inputcsv = 'todoist-python - Routines.csv'
    fpath_inputcsv = os.path.join(os.getcwd(), fname_inputcsv)

'todoist-python - Routine-test.csv'

# Functions

In [303]:
# creat label dictionary
def get_label_dict(todoist_api):
    api = TodoistAPI(todoist_api)
    try:
        labels = api.get_labels()
    except Exception as error:
        print(error)
    # label into dictionary
    label_dict = {}
    for i in range(len(labels)):
        label_dict[labels[i].name] = labels[i].id
    return label_dict



# creat project id dictionary
def get_project_dict(todoist_api):
    api = TodoistAPI(todoist_api)
    try:
        a = api.get_projects()
    except Exception as error:
        print(error)
    # label into dictionary
    d = {}
    for i in range(len(a)):
        d[a[i].name] = a[i].id
    return d



# creat section id dictionary from a project
def get_section_dict_pjc(todoist_api, project_id):
    api = TodoistAPI(todoist_api)
    try:
        a = api.get_sections(project_id=project_id)
    except Exception as error:
        print(error)
    # put into dictionary
    d = {}
    for i in range(len(a)):
        d[a[i].name] = a[i].id
    return d



# creat section id dictionaries
def get_section_dict(todoist_api):
    api = TodoistAPI(todoist_api)
    try:
        todoistpull = api.get_sections()
    except Exception as error:
        print(error)

    # put in dictionaries
    section_proj_dict = {}
    section_name_dict = {}

    for i in range(len(todoistpull)):
        section_proj_dict[todoistpull[i].id] = todoistpull[i].project_id
        section_name_dict[todoistpull[i].name] = todoistpull[i].id
    return section_proj_dict, section_name_dict


# get task dictionary by name
def get_task_dict(todoist_api):
    api = TodoistAPI(todoist_api)
    try:
        todoistpull = api.get_tasks()
    except Exception as error:
        print(error)

    # put in dictionaries
    task_dict = {}
    for i in range(len(todoistpull)):
        task_dict[todoistpull[i].content] = todoistpull[i].id
    return task_dict

In [281]:
# -- validation --
# this represents a list of current limitations
def validate_input_fields(todoist_dict, dfcsv):
    
    # limitation 1: all label, project, sections must pre-exist in Todoist
    fieldname_list = ['label','project','section']
    for fieldname in fieldname_list:
        entry_exist = []
        for x in dfcsv[fieldname].tolist():
            entry_exist.append(x in todoist_dict[fieldname].keys())
        
        if all(entry_exist):
            print(f'all {fieldname} found in Todoist')
        else:
            errormsg = f'Some {fieldname} entry does not exist in Todoist. Current code can not accomodate'
            raise Exception(errormsg)
    
    
    # limitation 2: only children level 2 can be handled
    if dfcsv.order[0] != 1 or any(dfcsv.order > 2):
        raise Exception('order number incorrect')

    
    # limitation 3: freq_offset must be entered, even for children task that uses days_from_parent as date calculations
    if any(dfcsv.freq_offset.isna()):
        raise Exception('freq_offset field must not be NaN')


    # limitation 4: all task must have time assigned
    if any(dfcsv.time.isna()):
        raise Exception('time field must not be NaN')

    # limitation 5: subtask backwards 27 days max
    if any(dfcsv.days_from_parent) < -28:
        raise Exception('code can not accomodate subtask backwards 27 days')

In [318]:
# deal with X weekday of the month
# get first month's third thu and next month's third thu and give the one that's after start date and before end date
def get_X_weekday_of_month(startdate, enddate, weeknum, freq_offset):
    thedate = []
    # get the first month's third thu
    d1 = pd.to_datetime(startdate) + pd.offsets.MonthBegin(n=-1)
    dlist = pd.date_range(d1, enddate, freq=freq_offset)
    dlist = dlist[weeknum-1] # 2 because the first value index is 0
    # check if is after start date
    if dlist >= startdate:
        thedate = dlist
    else:
        # get the second month's third thu
        d2 = pd.to_datetime(startdate) + pd.offsets.MonthBegin(n=1)
        dlist = pd.date_range(d2, enddate, freq=freq_offset)
        dlist = dlist[weeknum-1] # 2 because the first value index is 0
        if dlist <= enddate:
            thedate = dlist
        else:
            thedate = []
            raise Exception('no date within the specified range') 
    thedate = thedate.to_pydatetime()    
    print(thedate)      
    return thedate

In [327]:
# -- create date list from date requirements requirements --
# if weeknum is none, then deal with offset and freq normally
def get_datelist(startdate, enddate, tasknow):
    if tasknow.weeknum != 0:
        print('weeknum detected')
        # check if has weeknum off set, if yes, then need other calculation
        datelist = get_X_weekday_of_month(startdate, enddate, tasknow.weeknum, tasknow.freq_offset)
    else:
        print('weeknum not detected')
        if tasknow.weekstart == 0:
            datelist = pd.date_range((startdate), enddate, freq=tasknow.freq_offset)       
        elif tasknow.weekstart != 0:
            datelist = pd.date_range((startdate + pd.DateOffset(tasknow.weekstart)), enddate, 
                        freq=tasknow.freq_offset)
        else:
            raise Exception('weekstart entry invalid')
    return datelist

In [244]:
# -- build recurrent date + time --
def build_recurrent_datetime(datelist, tasknow_df,tasknow):
    # build date/time string
    datetimestr = []
    # convert to datetime format
    if type(datelist) is datetime.datetime:
        datetimestr.append(datelist.strftime("%Y-%m-%d") + ' ' + tasknow.time)
    else:
        datelist = datelist.to_pydatetime()
        for k in range(len(datelist)):
            a = datelist[k].strftime("%Y-%m-%d") + ' ' + tasknow.time
            datetimestr.append(a)
        
    # convert to datetime format
    datetimestr = pd.to_datetime(datetimestr)
    
    # convert to RC3339 format
    due_datetime = []
    for k in range(len(datetimestr)):
        due_datetime.append(datetimestr[k].isoformat('T'))
    # interim report 
    print(f'{len(due_datetime)} dates generated from freq offset "{tasknow.freq_offset}"')
    # add time series to dataframe
    tasknow_df['due_datetime'] = due_datetime
    return tasknow_df

In [95]:
# add task attributes
def add_task_attributes(tasknow, tasknow_df,todoist_dict):
    # -- build each task for todoist --
    # + duration to content
    content = f'{tasknow.content} [{tasknow.duration}m]'
    print(f'task name: {content}')
    tasknow_df['content'] = content

    # -- add the rest of task attributes --
    # add priority
    tasknow_df['priority'] = tasknow.priority
    # get section id
    sec_id = todoist_dict['section'][tasknow.section]
    # get section id to df
    tasknow_df['section_id'] = sec_id
    # get project id to df
    tasknow_df['project_id'] = todoist_dict['section_project'][sec_id]
    # translate label 
    # <can only take one label at a time, can expand to take multiple later>
    tasknow_df['label_ids'] = todoist_dict['label'][tasknow.label]
    return tasknow_df


In [89]:
# create tasks from list to Todoist
def create_task_from_tasknow_df(tasknow_df, todoist_api):
    for recur_num in range(len(tasknow_df)):
        api = TodoistAPI(todoist_api)
        try:
            task = api.add_task(
                    content=tasknow_df.content[recur_num],
                    due_lang='en',
                    project_id=int(tasknow_df.project_id[recur_num]),
                    section_id=int(tasknow_df.section_id[recur_num]),
                    due_datetime=tasknow_df.due_datetime[recur_num],
                    priority=int(tasknow_df.priority[recur_num]),
                    label_ids=[int(tasknow_df.label_ids[recur_num])],
            )
        except Exception as error:
            print(error)

Get Todoist API token

In [5]:
with open (todoist_token_fname) as f:
    todoist_api = f.readlines()
    todoist_api = todoist_api[0]

In [ ]:
# get todoist data
label_dict = get_label_dict(todoist_api)
proj_dict = get_project_dict(todoist_api)
section_proj_dict, section_name_dict = get_section_dict(todoist_api)

# make todoist dictionary
todoist_dict = {}
todoist_dict['label'] = label_dict
todoist_dict['project'] = proj_dict
todoist_dict['section'] = section_name_dict
todoist_dict['section_project'] = section_proj_dict

Code

In [16]:
# make start and end date timestamp
startdate = pd.to_datetime(startdate_string) 
enddate = startdate + pd.DateOffset(entryduration-1)
# report out
print(f'start date: {startdate}')
print(f'end date: {enddate}')
print(f'duration (days): {entryduration} days')

start date: 2022-08-22 00:00:00
end date: 2022-09-18 00:00:00
duration (days): 28 days


In [334]:
# import csv to deal wtih the data
dfcsv = pd.read_csv(fpath_inputcsv)
# validate entries
validate_input_fields(todoist_dict, dfcsv)


all label found in Todoist
all project found in Todoist
all section found in Todoist


# For each task

In [335]:
# process each task 
# rotate through each row
i = 15
# get task info
tasknow = dfcsv.loc[i,:]
display(tasknow)

# start a new dataframe for this task
tasknow_df = pd.DataFrame()

# get date list
datelist = get_datelist(startdate, enddate, tasknow)
# check if has time, if not, no date+time needed

# build recurrent date + time
tasknow_df = build_recurrent_datetime(datelist, tasknow_df, tasknow)
# add task attributes
tasknow_df = add_task_attributes(tasknow, tasknow_df, todoist_dict)
tasknow_df

content                  Allyship meeting
duration                               60
date                               16-Aug
time                                13:00
freq_offset                         W-TUE
order                                   1
days_from_parent                        0
offset_start                            0
weeknum                                 3
recurrent_string    3rd Tues of the month
priority                                4
project                              Test
section                        Work stuff
label                            Computer
Unnamed: 14                           NaN
Unnamed: 15                           NaN
Unnamed: 16                           NaN
Unnamed: 17                           NaN
Unnamed: 18                           NaN
Name: 15, dtype: object

weeknum detected


IndexError: index 2 is out of bounds for axis 0 with size 2

In [293]:
# -- deal with children task -- (option 1)
# order - check if this task belong to a parent task 
if i != (len(dfcsv)-1): # if this task is not he last task
    if dfcsv.loc[i+1,'order'] !=1: # if the next task is children 
        # find index to last children task
        order_is_1 = dfcsv.loc[i+1:len(dfcsv), 'order'] == 1 # see which subsquent tasks are parent (order = 1)
        # see if any parent task left
        if sum(order_is_1) == 0: # if no more parent task left
            ind_last_children = len(dfcsv)-1
        else:
            ind_last_children = min(order_is_1.index[order_is_1.tolist()]) - 1
        
        children_task_dict = {} # create children task dictionary
        for j in range(i+1,ind_last_children): # loop through children task
            print(dfcsv.loc[j,'content']) # print children task content
            df = pd.DataFrame() # create data frame
            # build df for each children task
            
            # store the task number for the upcoming children task
            # calculate dates based on parent task


Prep Allyship meeting
Allyship impact note
Determine allyship topic
Finalize Allyship content


In [308]:
tasknow_df.due_datetime


0    2022-08-25T13:00:00
1    2022-09-01T13:00:00
2    2022-09-08T13:00:00
3    2022-09-15T13:00:00
Name: due_datetime, dtype: object

[]


In [102]:



# create tasks from list to Todoist
create_task_from_tasknow_df(tasknow_df, todoist_api)


2 dates generated from freq offset "2W-THU"
task name: SC Biweekly Lead Sync [45m]


4 dates generated from freq offset "W-SAT"
task name: Weekend Beauty Routine [120m]


There are problem with the project id / section id dtypes